# Cours TAL – Labo 5
Nathan Gonzalez Montes et Vincent Guidoux

## Exercice 1 - Tester et évaluer un modèle entraîné sur Google News

### a) Installez la version 3.7.1 de gensim, la bibliothèque implémentant les outils pour travailler avec Word2Vec (pip install --upgrade gensim). Puis chargez le modèle word2vec pré-entraîné sur le corpus Google News en écrivant : w2v_model = gensim.downloader.load("word2vecgoogle-news-300"), ce qui télécharge le fichier la première fois, et enfin en ne gardant que les vecteurs de mots, avec « w2v_vectors = w2v_model.wv » puis « del w2v_model »). Pour éviter ce téléchargement, on fournit le fichier sur une clé USB, et on peut le charger avec w2v_vectors = KeyedVectors.load_word2vec_format(path_to_file, binary=True). 

#### Importation des librairies nécessaires

In [1]:
import numpy
import nltk
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

#### Chargement du fichier

In [2]:
w2v_vectors = KeyedVectors.load_word2vec_format("word2vec-google-news-300.gz", binary=True)

#### Quelle place occupe le processus noyau en mémoire une fois les vecteurs de mots chargés ?

4.6 Go

### b. Quelle est la dimension de l’espace vectoriel dans lequel les mots sont représentés ? Et quelle est la taille du vocabulaire du modèle ? Montrez 5 mots qui sont dans le vocabulaire et un qui ne l’est pas.

#### Quelle est la dimension de l’espace vectoriel dans lequel les mots sont représentés ?

In [3]:
w2v_vectors.vectors.shape

(3000000, 300)

#### Et quelle est la taille du vocabulaire du modèle ?

In [4]:
len(w2v_vectors.vocab)

3000000

#### Montrez 5 mots qui sont dans le vocabulaire et un qui ne l’est pas.

In [5]:
words = ["victory", "defeat", "win", "loose", "champion", "Echallens"]

for word in words:
    if word in w2v_vectors.vocab:
        print("{} is there".format(word))
    else:
        print("{} is not there".format(word))

victory is there
defeat is there
win is there
loose is there
champion is there
Echallens is not there


### c. Comment peut-on mesurer la distance entre deux mots dans cet espace ? Calculez la distance entre les mots rabbit et carrot.

In [6]:
distance = w2v_vectors.distance("rabbit", "carrot")
print("{:.3f}".format(distance))

0.637


In [7]:
similarity = w2v_vectors.similarity("rabbit", "carrot")
print("{:.3f}".format(similarity))

0.363


On peut observer que les méthodes distance et similarty son complémentaires. La somme des 2 valeurs est égale à 1.

### d. Testez le modèle de distance entre mots. Est-ce que des mots proches sémantiquement sont aussi proches dans l’espace vectoriel, et inversement ? Testez au moins cinq paires de mots.

In [8]:
# On teste les similarity, ne devrait-il pas être les distances ? - Guidoux to Gonzalez

similarity = w2v_vectors.similarity("victory", "triumph") 
print("1. {:.3f}".format(similarity))

similarity = w2v_vectors.similarity("victory", "win")
print("2. {:.3f}".format(similarity))

similarity = w2v_vectors.similarity("victory", "champion")
print("3. {:.3f}".format(similarity))

similarity = w2v_vectors.similarity("victory", "victories")
print("4. {:.3f}".format(similarity))

similarity = w2v_vectors.similarity("victory", "defeat")
print("5. {:.3f}".format(similarity))

similarity = w2v_vectors.similarity("victory", "lose")
print("6. {:.3f}".format(similarity))

similarity = w2v_vectors.similarity("dog", "cat")
print("7. {:.3f}".format(similarity))

similarity = w2v_vectors.similarity("car", "bicycle")
print("8. {:.3f}".format(similarity))

similarity = w2v_vectors.similarity("Vaud", "Yverdon")
print("9. {:.3f}".format(similarity))

1. 0.829
2. 0.802
3. 0.390
4. 0.753
5. 0.691
6. 0.161
7. 0.761
8. 0.536
9. 0.454


On peut observer que les mots proches sémantiquemenet sont aussi proches dans l'espace et que la similarité est assez grande, comme par exemple **victory** and **triumph** (des synonymes), ou **victory** et **defeat** (des antonymes). 

Par contre, on peut aussi observer que des mots qui sont proches sémantiquement (**victory** et **champion**), sont assez éloignés quand on peut dire qu'une victoire mène à avoir un champion.

### e. Y a-t-il des cas ambigus, et pourquoi selon vous ? Par exemple, des mots opposés selon le sens sont-ils proches ou éloignés dans l’espace ?

On peut dire que oui en voyant les cas de l'exercise précédent avec par exemple, le fait que **victory** et **defeat** sont **similaires** à **69,1%** quand ce sont des mots opposés, mais vu qu'une victoire peut mener une défaite, la **distance** entre les deux mots ne devrait pas être très grande, ce qui serait plus logique comme résultat (vu que le complémentaire à cette similarité entre les mots est la distance qui les séparent). Tandis que la similarité entre **victory** et **lose** est très basse, car ce sont des mots opposés.

Par rapport à la distance, si on compare **victory** et **champion**, ils sont assez distant si l'on peut considérer que si on a une victoire, il y a forcément un champion, et donc celui-ci devrait être plus proche du mot **victory**.

On peut voir aussi que **dog** et **cat** son similaires entre eux quand ce sont des animaux différents.

### f. Que dire des mots ayant plusieurs sens ? Pouvez-vous donner 3 exemples de ce problème ?

In [9]:
distance = w2v_vectors.distance("present", "past")
print("Opposite (distance): {:.3f}".format(distance))
distance = w2v_vectors.distance("present", "now")
print("Same (distance): {:.3f}".format(distance))
distance = w2v_vectors.distance("present", "gift")
print("Same (distance): {:.3f}".format(distance), "\n")

similarity = w2v_vectors.similarity("present", "past")
print("Opposite (similarity): {:.3f}".format(similarity))
similarity = w2v_vectors.similarity("present", "now")
print("Same (similarity): {:.3f}".format(similarity))
similarity = w2v_vectors.similarity("present", "gift")
print("Same (similarity): {:.3f}".format(similarity), "\n")

distance = w2v_vectors.distance("right", "left")
print("Opposite (distance): {:.3f}".format(distance))
distance = w2v_vectors.distance("right", "correct")
print("Same (distance): {:.3f}".format(distance))
distance = w2v_vectors.distance("right", "law")
print("Same (distance): {:.3f}".format(distance), "\n")

similarity = w2v_vectors.similarity("right", "left")
print("Opposite (similarity): {:.3f}".format(similarity))
similarity = w2v_vectors.similarity("right", "correct")
print("Same (similarity): {:.3f}".format(similarity))
similarity = w2v_vectors.similarity("right", "law")
print("Same (similarity): {:.3f}".format(similarity))

Opposite (distance): 0.891
Same (distance): 0.856
Same (distance): 0.814 

Opposite (similarity): 0.109
Same (similarity): 0.144
Same (similarity): 0.186 

Opposite (distance): 0.508
Same (distance): 0.597
Same (distance): 0.885 

Opposite (similarity): 0.492
Same (similarity): 0.403
Same (similarity): 0.115


Ici, on a utilisé **present** comme mot avec différent sens (comme **cadeau** et comme le **présent**), et on les a comparé, pour le **présent**, à un **synonyme et antonyme**, et pour **cadeau**, à un **synonyme**.

On peut voir que étant un mot à plusieurs sens peut poser problème car même les synonymes sont distants entre eux et ne sont donc pas très similaires, mais avec l'antonyme, on a pareil, ce qui n'est pas très logique.

Mais par exemple, pour le mot **right**, il y a des résultats partagés, avec son opposé **left**, qui est presque à **50-50%**, et un synonyme comme **correct** qui est un peu plus distant et moins similaire, donc à **60-40%**. Tandis que si on le compare à quelque chose qui est proche syntaxiquement comme **law** (les droits et la loi), la distance s'aggrandi et il n'y a quasi pas de similarité.

### g. En vous aidant de la documentation de Gensim sur KeyedVectors, mesurez de manière quantitative la performance du modèle sur le corpus WordSimilarity-353. Expliquez ce que signifient vos résultats.

evaluate_word_pairs computes correlation of the model with human similarity judgments.

In [10]:
from gensim.test.utils import datapath
similarities = w2v_vectors.evaluate_word_pairs(datapath('wordsim353.tsv'))



In [11]:
print("Pearson correlation coefficient with 2-tailed p-value : ", similarities[0])
print("\n")
print("Spearman rank-order correlation coefficient between the similarities from the dataset and the similarities produced by the model itself, with 2-tailed p-value : ", similarities[1])
print("\n")
print("The ratio of pairs with unknown words : ", similarities[2])

Pearson correlation coefficient with 2-tailed p-value :  (0.6238773444802619, 1.7963251834702263e-39)


Spearman rank-order correlation coefficient between the similarities from the dataset and the similarities produced by the model itself, with 2-tailed p-value :  SpearmanrResult(correlation=0.6589215888009288, pvalue=2.5346056459149263e-45)


The ratio of pairs with unknown words :  0.0


### h. De même, en vous inspirant de la documentation, évaluez le modèle sur les données de test appelées questions-words.txt. Pouvez-vous expliquer ce que mesure ce test ? Les résultats du modèle sont-ils satisfaisants ? Commentez.

evaluate_word_analogies compute performance of the model on an analogy test set

In [12]:
analogy_scores = w2v_vectors.evaluate_word_analogies(datapath('questions-words.txt'))

In [18]:
print("The overall evaluation score on the entire evaluation set : ", analogy_scores[0])
print("\n")
print("Results broken down by each section of the evaluation set. Each dict contains the name of the section under the key ‘section’, and lists of correctly and incorrectly predicted 4-tuples of words under the keys ‘correct’ and ‘incorrect’. : ")
print("\n")

for section in analogy_scores[1]:
    print("section \"{section}\" : ".format(section=section["section"]))
    print("number of correct : ", len(section["correct"]))
    print("number of incorrect : ", len(section["incorrect"]))
    print("\n")

The overall evaluation score on the entire evaluation set :  0.7401448525607863


Results broken down by each section of the evaluation set. Each dict contains the name of the section under the key ‘section’, and lists of correctly and incorrectly predicted 4-tuples of words under the keys ‘correct’ and ‘incorrect’. : 


section "capital-common-countries" : 
number of correct :  421
number of incorrect :  85


section "capital-world" : 
number of correct :  3552
number of incorrect :  816


section "currency" : 
number of correct :  230
number of incorrect :  578


section "city-in-state" : 
number of correct :  1779
number of incorrect :  688


section "family" : 
number of correct :  436
number of incorrect :  70


section "gram1-adjective-to-adverb" : 
number of correct :  290
number of incorrect :  702


section "gram2-opposite" : 
number of correct :  353
number of incorrect :  459


section "gram3-comparative" : 
number of correct :  1216
number of incorrect :  116


section "gra

## Exercice 2 - Entraîner et tester deux nouveaux modèles à partir de corpus

### a. Afin de pouvoir télécharger un corpus textuel par Gensim, exécutez la commande : import gensim.downloader. Puis récupérez le corpus des 108 premiers caractères de Wikipédia (en anglais) avec la commande : corpus = gensim.downloader.load('text8'). Combien de phrases et de mots possède ce corpus ?

In [14]:
import gensim.downloader
corpus = gensim.downloader.load('text8')

### b. Entraînez un nouveau modèle word2vec sur ce nouveau corpus. Combien de temps prend l’entraînement et quelle est la taille (en Mo ou Go) du modèle word2vec résultant ?

### c. Quelle dimension avez-vous choisi pour le plongement (embedding) de ce nouveau modèle ?

### d. Testez quantitativement la qualité de ce nouveau modèle comme dans la partie 1 (points h et i). Ce modèle est-il meilleur que celui entraîné sur Google News ? Quelle serait la raison de la différence ?

### e. Considérez maintenant le corpus quatre fois plus grand constitué de la concaténation du corpus text8 et des dépêches économiques de Reuters (corpus que nous vous mettons à disposition en ligne ou sur la clé USB comme wikipedia_augmented.dat). Utilisez ce nouveau corpus afin d’entraîner un nouveau modèle word2vec. Vous devez pour cela utiliser la fonction Text8Corpus() afin de charger et pré-traiter le texte du corpus (tokenization et segmentation en phrases, qui étaient déjà faites par la fonction load() du downloader). Combien de temps prend l’entraînement et quelle est la taille (en Mo) du modèle word2vec résultant ? Quelle est la dimension de l’embedding ?

### f. Testez aussi ce modèle. Est-il meilleur que le précédent ? Pour quelle raison ?

### g. Créez un nouveau fichier de test en augmentant questions-words.txt avec une catégorie de mots de votre choix (environ 20 items de test). Par exemple, à partir de {(eye, see), (ear, listen), (foot, walk)} on peut construire 6 items de test. Testez les trois modèles sur ces données, présentez et commentez vos résultats.